In [3]:
import pandas as pd
#import matplotlib.pyplot as plt
import json, urllib.request
from pandas.io.json import json_normalize
import NISTISODB as iso
import timeit
import requests
from tqdm import tqdm, tqdm_notebook
import numpy as np

In [2]:
# Open datafile and readout data to list
json_url = urllib.request.urlopen('https://adsorbents.nist.gov/isodb/api/isotherms.json')
#json_url = urllib.request.urlopen('https://adsorbents.nist.gov/isodb/api/isotherm/10.1002adfm.201101479.Isotherm1.json')
data = json.loads(json_url.read())


In [6]:
df=pd.DataFrame.from_dict(data).drop(columns=['category','isotherm_type'])
df.shape

(32552, 5)

In [3]:
#Change datalist to dataframe and remove empty columns 'category' and 'isotherm_type'
df=pd.DataFrame.from_dict(data).drop(columns=['category','isotherm_type'])

#Adsorbates column entries are changed via following process : [dict]->dict->value 
df.adsorbates=pd.DataFrame(df.adsorbates.tolist(), columns=['adsorbates','team2','team1','team2']).adsorbates.apply(pd.Series)

#Adsorbent column entries are changed via following process: dict->value
df.adsorbent=df.adsorbent.apply(pd.Series)

#Change temperature from kelvin to celsius, add .json to filename column, and rename select columns to more descriptive names
df['temperature']=df.temperature.apply(lambda x: x - 273)
df['filename']=df.filename.apply(lambda x: x + ".json")

df=df.rename(columns={"adsorbates": "adsorbates_InChIkey", "adsorbent": "adsorbent_hashkey","temperature":"temp_C","filename":"isotherm_file"})
                                     

KeyboardInterrupt: 

In [ ]:
#CO2df contains entries in which CO2 is an adsorbate, the adsorbate column is also removed
CO2df=df[df.adsorbates_InChIkey=='CURLTUGMZLYLDI-UHFFFAOYSA-N'].drop(columns=['adsorbates_InChIkey'])
#CO2df.temp_C.hist()
CO2df.head()

In [142]:
#Class takes isotherm file and then returns various units from isotherm file
class poop:
    def __init__(self,filename):
        host='adsorption.nist.gov'
        URL = 'https://'+host+'/isodb/api/isotherm/'+filename
        isotherm_JSON = json.loads(requests.get(URL).content)
        #----------------------------------------------------------------
        #Essential Check: Confirm that input isotherm is single component
        #n_adsorbates = len(isotherm_JSON["adsorbates"])
        self.adsorbent = isotherm_JSON["adsorbent"]
        #self.adsorbate = isotherm_JSON["adsorbates"][0]
        #self.temperature = float(isotherm_JSON["temperature"])
        #if n_adsorbates > 1:
         #   self.pressureUnits='Multicomponent'
         #   self.adsorptionUnits='Multicomponent'
            
            #raise Exception("ERROR: Filename "+filename+" has multiple adsorbates.\n Input isotherm must be single component.")
        #----------------------------------------------------------------
        #else:
            #self.pressureUnits = isotherm_JSON["pressureUnits"]
            #self.adsorptionUnits = isotherm_JSON["adsorptionUnits"]
            #self.p_label = "Pressure ("+self.pressureUnits+")"
            #self.ads_label = "Loading ("+self.adsorptionUnits+")"
            #self.p_list = [ point["pressure"] for point in isotherm_JSON["isotherm_data"] ]
            # Uses the first entry (index 0) as the isotherm is single component, or would have failed earlier test 
            #self.ads_list = [ point["species_data"][0]["adsorption"] for point in isotherm_JSON["isotherm_data"] ]

In [18]:

#Adds the pressure units into the COdf
CO2df['P_units']=CO2df.isotherm_file.apply(lambda x:poop(x).pressureUnits)

#Alternative process conducts punit and adsunit as once this was never used
##pd.concat(CO2df,df.isotherm_file.apply(lambda x: ({'P_units':poop(s).pressureUnits, 'ads_units':poop(s).adsorptionUnits})))
#Save the CO2df to csv
CO2df.to_csv(r'./CO2df.csv', index=True)

NameError: name 'requests' is not defined

In [22]:
#reload from previous cell the CO2df
CO2df=pd.read_csv('./CO2df.csv')
#loading bar
tqdm.pandas()
#adds adsorption units to CO2df
CO2df['ads_units']=CO2df.isotherm_file.progress_apply(lambda x:poop(x).adsorptionUnits)
#data was saved to csv
CO2df.to_csv(r'./CO2df.csv', index=True)

100%|█████████████████████████████████████████| 7190/7190 [53:17<00:00,  2.19it/s]


In [151]:
#adds actual adsorbent name to pandas 
tqdm_notebook().pandas()
CO2df['adsorbent']=CO2df.isotherm_file.progress_apply(lambda x:poop(x).adsorbent.get('name'))
#data was saved to csv
CO2df.to_csv(r'./CO2df.csv', index=True)

In [4]:
CO2df=pd.read_csv('./CO2df.csv')
CO2df=CO2df.drop(columns='Unnamed: 0')
CO2df=CO2df.rename(columns={"Unnamed: 0.1": "Original_idx"})
CO2df['ads_units'].value_counts()

mmolgdf=CO2df[CO2df['ads_units']== 'mmol/g']

In [5]:
mmolgdf

,Original_idx,DOI,adsorbent_hashkey,isotherm_file,temp_C,P_units,ads_units,adsorbent
27,260,10.1002/adfm.201502069,NIST-MATDB-a8881f649c6a1c8dc5760b668d3fc1f4,10.1002adfm.201502069.Isotherm10.json,30,bar,mmol/g,DArE@PAF-1 50wt%(Photoswitched)
28,265,10.1002/adfm.201502069,NIST-MATDB-fcbfa05a72a31364cffdbc3fcb55d072,10.1002adfm.201502069.Isotherm6.json,30,bar,mmol/g,PAF-1(Photoswitched)
29,266,10.1002/adfm.201502069,NIST-MATDB-e444d66fc68bec5534b2ef85f89a4145,10.1002adfm.201502069.Isotherm7.json,30,bar,mmol/g,DArE@PAF-1 1wt%(Photoswitched)
30,267,10.1002/adfm.201502069,NIST-MATDB-c354a25737a46d160dbcebe6d23f72ee,10.1002adfm.201502069.Isotherm8.json,30,bar,mmol/g,DArE@PAF-1 5wt%(Photoswitched)
31,268,10.1002/adfm.201502069,NIST-MATDB-0c72526b865dae77fe632a90a7496e7c,10.1002adfm.201502069.Isotherm9.json,30,bar,mmol/g,DArE@PAF-1 20wt%(Photoswitched)
...,...,...,...,...,...,...,...,...
7185,30715,10.8888/s0167-2991(08)63564-8,NIST-MATDB-453276ee992ed07dc207ed08b2b221ce,10.8888s0167-2991(08)63564-8.Isotherm2.json,30,bar,mmol/g,Carbon
7186,30718,10.8888/s10450-012-9407-1,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,10.8888s10450-012-9407-1.Isotherm1.json,25,bar,mmol/g,ZIF-8
7187,30719,10.8888/s10450-012-9407-1,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,10.8888s10450-012-9407-1.Isotherm11.json,25,bar,mmol/g,ZIF-8
7188,30720,10.8888/s10450-012-9407-1,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,10.8888s10450-012-9407-1.Isotherm13.json,35,bar,mmol/g,ZIF-8


In [4]:

mmolgdf.shape[0]

arg=[]
arg=poop(mmolgdf.iloc[364].isotherm_file)
argg=np.array(arg.p_list)
argp=np.array(arg.ads_list)
import matplotlib.pyplot as plt
plt.scatter(argg,argp)
argg


NameError: name 'poop' is not defined

In [15]:
#Adds CO2 capacities into CO2cap array for units of mmolgdf
CO2cap=np.zeros(mmolgdf.shape[0])
for i in tqdm_notebook(range(mmolgdf.shape[0])):
    arg=poop(mmolgdf.iloc[i].isotherm_file)
    argg=np.array(arg.p_list)
    argp=np.array(arg.ads_list)
    if np.any(argg < 0.04)== False:
        argg=np.append(np.zeros(1),argg)
        argp=np.append(np.zeros(1),argp)
    else:
        pass
    if np.any(argg>0.04) == False:
        CO2cap[i]=np.nan
    else:
        maxPval, minPval=argg[argg > 0.04].min(),argg[argg < 0.04].max()
        #idx=np.array([np.where(argg==minval)[0][0],np.where(argg==maxval)[0][0]])
        #rvals(Pvals first row,Cap vals second row)
        rvals=np.array([[minPval, maxPval], [argp[np.where(argg==minPval)][0],argp[np.where(argg==maxPval)][0]]])
        slope=(rvals[1,0]-rvals[1,1])/(rvals[0,0]-rvals[0,1])
        CO2cap[i]=slope*0.04+(rvals[1,0]-(slope*rvals[0,0]))

In [6]:
mmolgdf['CO2_capacity (mmol/g)']=CO2cap
#mmolgdf=mmolgdf.drop(columns=['adsorbent_hashkey'])
mmolgdf.to_csv(r'./CO2capmmolg.csv', index=True)
mmolgdf.sort_values(by='CO2_capacity (mmol/g)', ascending=False)
acsmof=mmolgdf[mmolgdf['temp_C']>0 ].sort_values(by='CO2_capacity (mmol/g)', ascending=False )

NameError: name 'CO2cap' is not defined

In [7]:
mmolgdf

,Unnamed: 0,Unnamed: 0.1,DOI,adsorbent_hashkey,isotherm_file,temp_C,P_units,ads_units,CO2cap
0,27,260,10.1002/adfm.201502069,NIST-MATDB-a8881f649c6a1c8dc5760b668d3fc1f4,10.1002adfm.201502069.Isotherm10.json,30,bar,mmol/g,0.000535
1,28,265,10.1002/adfm.201502069,NIST-MATDB-fcbfa05a72a31364cffdbc3fcb55d072,10.1002adfm.201502069.Isotherm6.json,30,bar,mmol/g,0.000535
2,29,266,10.1002/adfm.201502069,NIST-MATDB-e444d66fc68bec5534b2ef85f89a4145,10.1002adfm.201502069.Isotherm7.json,30,bar,mmol/g,0.000535
3,30,267,10.1002/adfm.201502069,NIST-MATDB-c354a25737a46d160dbcebe6d23f72ee,10.1002adfm.201502069.Isotherm8.json,30,bar,mmol/g,0.000535
4,31,268,10.1002/adfm.201502069,NIST-MATDB-0c72526b865dae77fe632a90a7496e7c,10.1002adfm.201502069.Isotherm9.json,30,bar,mmol/g,0.000535
5,37,293,10.1002/adma.200602645,NIST-MATDB-7cb7f2ebc677a987561f61f34f01f965,10.1002adma.200602645.Isotherm1.json,31,bar,mmol/g,0.000535
6,56,361,10.1002/adsu.201700092,NIST-MATDB-86f0ec02124e8670e186f3e87a707e89,10.1002adsu.201700092.Isotherm11.json,25,bar,mmol/g,0.000535
7,57,362,10.1002/adsu.201700092,NIST-MATDB-d6934b6bfd67e4cc829e07b7d1e22e49,10.1002adsu.201700092.Isotherm12.json,25,bar,mmol/g,0.000535
8,58,363,10.1002/adsu.201700092,NIST-MATDB-aa8168e0eaccc317f0e4d55f6cd58d20,10.1002adsu.201700092.Isotherm13.json,25,bar,mmol/g,0.000535
9,84,455,10.1002/Aic.11865,NIST-MATDB-1b4fff6cb48e3c544cb81776c1f1ac76,10.1002Aic.11865.isotherm3.json,25,bar,mmol/g,0.000535


In [7]:
#For each column entry of CO2 df we should add isotherm data and adsorption value at 400 ppm CO2
# (1) Call isotherm data; (2a) calculate 400 ppm capacity by taking points the two points closest to 400 ppm from the left and the right
#(2b)calculate 400 ppm capacity by fitting isotherm
#accuracy checks, what is min pressure?
#add adslist and plist into mmolgdf
#need to find nearest value from 0.04 bar run CO2capacity=(y2-y1/x2-x1)*0.04+(y1-m*x1)
CO2df=pd.read_csv('./CO2df.csv')
mmolgdf=pd.read_csv('./CO2capmmolg.csv')

In [37]:
mmolgdf['CO2_capacity (mmol/g)']

0       0.000535
1       0.021132
2       0.037374
3       0.027828
4       0.018697
          ...   
3835    0.173369
3836    0.072778
3837    0.023226
3838    0.014146
3839    0.012373
Name: CO2_capacity (mmol/g), Length: 3840, dtype: float64

In [34]:
tempdf=mmolgdf[~mmolgdf['adsorbent'].isin(['Mg2(dobdc)','Mg-MOF-74','CPO-27-Mg'])][mmolgdf['temp_C']>0].sort_values(by='CO2_capacity (mmol/g)', ascending=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [32]:
tempdf=mmolgdf[mmolgdf['temp_C']>0].sort_values(by='CO2_capacity (mmol/g)', ascending=False)

In [35]:
tempdf.iloc[0:30]

,Unnamed: 0,DOI,isotherm_file,temp_C,adsorbent,CO2_capacity (mmol/g)
532,1039,10.1016/j.cej.2012.04.071,10.1016j.cej.2012.04.071.isotherm3.json,15,Cr-MIL(101),36.426320
2957,5431,10.1039/c1ee01380g,10.1039c1ee01380g.Isotherm3.json,25,CAU-1,34.487819
2958,5432,10.1039/c1ee01380g,10.1039c1ee01380g.Isotherm4.json,50,CAU-1,15.380033
3388,6233,10.1039/C3ta11548h,10.1039C3ta11548h.isotherm12.json,22,InOF-2-Li+,5.800698
2536,4658,10.1021/Jz3008485,10.1021Jz3008485.Isotherm6.json,25,CuBTC,5.730934
1136,2022,10.1016/j.micromeso.2013.06.023,10.1016j.micromeso.2013.06.023.Isotherm12.json,25,Mg-MOF-74 (4),5.325166
2603,4750,10.1021/La205152f,10.1021La205152f.Isotherm1.json,25,Al-rho-MOF,4.879541
3385,6218,10.1039/C3sc51319j,10.1039C3sc51319j.isotherm8.json,5,CPO-27-Ni,4.634262
1135,2021,10.1016/j.micromeso.2013.06.023,10.1016j.micromeso.2013.06.023.Isotherm11.json,25,Mg-MOF-74 (3),4.545112
3700,6904,10.1080/08927022.2011.602974,10.108008927022.2011.602974.Isotherm18.json,25,Al-Rho-ZMOF,4.509279


In [29]:
tempdf.shape

(101, 6)

In [24]:
tempdf[['CO2_capacity (mmol/g)','adsorbent']].iloc[100:350,:]

,CO2_capacity (mmol/g),adsorbent
3447,2.108006,MIL-101(Cr)-250nm-PEI-300
2053,2.107669,Zeolite 5A
2875,2.104671,Rho-ZMOF
2070,2.103573,Zeolite 13X
2826,2.087448,Mg2(dobpdc)
...,...,...
2196,0.744255,Cu-TDPAT
3077,0.741932,C21H31NO20P2Zn3
2552,0.734895,Pd(2-pymo)2
2436,0.731270,Fe-MOF-74


In [9]:
CO2list=[]
CO2adsorblist=[]
N2adsorblist=[]
N2list=[]
CO2DOIlist=[]
N2DOIlist=[]
for x in range(len(data)):
    t=data[x]
    if t['adsorbates']==[{'InChIKey':'CURLTUGMZLYLDI-UHFFFAOYSA-N'}]:
        CO2adsorblist.append(t['adsorbent'])
        CO2DOIlist.append(t['DOI'])
        CO2list.append(data[x])
    if t['adsorbates']==[{'InChIKey':'IJGRMHOSHXDMSA-UHFFFAOYSA-N'}]:
        if t['temperature']>77:
            N2list.append(data[x])
            N2adsorblist.append(t['adsorbent'])
            N2DOIlist.append(t['DOI'])

In [10]:
CO2 = [] 
N2 = [] 
DOIN2= []
DOICO2= []
[CO2.append(x) for x in CO2adsorblist if x not in CO2] 
[N2.append(x) for x in N2adsorblist if x not in N2] 
[DOICO2.append(x) for x in CO2DOIlist if x not in DOICO2] 
[DOIN2.append(x) for x in N2DOIlist if x not in DOIN2] 
common = [i for i in CO2 if i in N2]
commonDOI= [i for i in DOICO2 if i in DOIN2]
#513 sorbents that contain both CO2 and N2 isotherm data
#362 paper

In [11]:
CO2list

[{'filename': '10.1002adfm.201101479.Isotherm13',
  'DOI': '10.1002/adfm.201101479',
  'adsorbent': {'hashkey': 'NIST-MATDB-655315f090663b4723680a6683f06403'},
  'adsorbates': [{'InChIKey': 'CURLTUGMZLYLDI-UHFFFAOYSA-N'}],
  'category': '',
  'temperature': 288,
  'isotherm_type': ''},
 {'filename': '10.1002adfm.201101479.Isotherm14',
  'DOI': '10.1002/adfm.201101479',
  'adsorbent': {'hashkey': 'NIST-MATDB-655315f090663b4723680a6683f06403'},
  'adsorbates': [{'InChIKey': 'CURLTUGMZLYLDI-UHFFFAOYSA-N'}],
  'category': '',
  'temperature': 298,
  'isotherm_type': ''},
 {'filename': '10.1002adfm.201101479.Isotherm15',
  'DOI': '10.1002/adfm.201101479',
  'adsorbent': {'hashkey': 'NIST-MATDB-655315f090663b4723680a6683f06403'},
  'adsorbates': [{'InChIKey': 'CURLTUGMZLYLDI-UHFFFAOYSA-N'}],
  'category': '',
  'temperature': 303,
  'isotherm_type': ''},
 {'filename': '10.1002adfm.201101479.Isotherm16',
  'DOI': '10.1002/adfm.201101479',
  'adsorbent': {'hashkey': 'NIST-MATDB-655315f090663b4

In [12]:
common[0]

{'hashkey': 'NIST-MATDB-655315f090663b4723680a6683f06403'}

In [13]:
finlist=[]
clist=[]
nlist=[]
DOIlist=[]
NDOIlist=[]
for i in range(len(common)):
    a=common[i]
    for x in range(len(data)):       
        t=data[x]
        #print(t['DOI'])
        if t['adsorbent']==common[i]:
            #print(t['adsorbates'])
            if t['adsorbates']==[{'InChIKey':'CURLTUGMZLYLDI-UHFFFAOYSA-N'}]:
                clist.append(t['temperature'])
                DOIlist.append(t['DOI'])
             #   print(clist)
            if t['adsorbates']==[{'InChIKey':'IJGRMHOSHXDMSA-UHFFFAOYSA-N'}]:
                if t['temperature']>77:
                    nlist.append(t['temperature'])
                    NDOIlist.append(t['DOI'])
              #      print(nlist)
            
            #print(DOI)
    finlist.append([a['hashkey'],DOIlist,clist,NDOIlist,nlist])
    clist=[]
    nlist=[]
    DOIlist=[]
    NDOIlist=[]

In [14]:
df=pd.DataFrame(finlist)
df

,0,1,2,3,4
0,NIST-MATDB-655315f090663b4723680a6683f06403,"[10.1002/adfm.201101479, 10.1002/adfm.20110147...","[288, 298, 303, 308, 273]",[10.1002/adfm.201101479],[298]
1,NIST-MATDB-8fad47f65fa08c7ac5157aa92c218b1d,"[10.1002/adfm.201101479, 10.1002/adfm.20110147...","[288, 298, 273]",[10.1002/adfm.201101479],[298]
2,NIST-MATDB-d049583f6d607e59dc804c4029176dcb,"[10.1002/adfm.201101479, 10.1002/adfm.20110147...","[288, 298, 273]",[10.1002/adfm.201101479],[298]
3,NIST-MATDB-b16ee19c4a2d5d7d2f7e09cb7263392f,"[10.1002/adfm.201101479, 10.1002/adfm.20110147...","[288, 298, 273]",[10.1002/adfm.201101479],[298]
4,NIST-MATDB-1b5c20248717ca46ef341e3c5f2c1f30,"[10.1002/adfm.201200084, 10.1002/adfm.20120008...","[273, 298, 298, 283, 257, 263, 273, 298, 303, ...","[10.1002/adfm.201200084, 10.1002/adfm.20120008...","[273, 298, 298, 283, 283]"
5,NIST-MATDB-adac5c7a8c6d4ec0e904766ae85d617d,"[10.1002/adma.200601983, 10.1002/adma.200601983]","[196, 298]",[10.1002/adma.200601983],[87]
6,NIST-MATDB-081a0818678716f502aee0b58ac1b30a,"[10.1002/adma.200601983, 10.1016/j.apsusc.2009...","[196, 298, 298, 298, 298, 297, 297, 303]","[10.1002/adma.200601983, 10.1021/Jp0012101, 10...","[87, 283, 283]"
7,NIST-MATDB-5f0d8d1b40eea63c05e68d3973285eae,"[10.1002/adma.200601983, 10.1002/adma.200601983]","[196, 298]",[10.1002/adma.200601983],[87]
8,NIST-MATDB-7cb7f2ebc677a987561f61f34f01f965,"[10.1002/adma.200602645, 10.1002/anie.20080306...","[304, 300, 300, 230, 230, 300, 300, 300, 304, ...",[10.1021/Ja803899q],[303]
9,NIST-MATDB-991daf7313251e7e607e2bab2da57e33,"[10.1002/adma.201400428, 10.1002/adma.20140382...","[298, 295, 295, 313, 303, 303, 313, 308, 308, ...","[10.1002/Aic.11298, 10.1002/Aic.11298, 10.1002...","[295, 295, 313, 298, 298, 298, 348, 298, 323, ..."


In [15]:
a=common[1]
obj=a['hashkey']
list=[obj,doi,co2t,n2t]


NameError: name 'doi' is not defined

In [ ]:
newlist=[]
list=['1','24','sdaf']
for x in list:
    if x=='1':
        newlist.append(x)
t=data[0]
t['adsorbates']==[{'InChIKey': 'UFHFLCQGNIYNRP-UHFFFAOYSA-N'}]